In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

In [3]:
sentences = ['안녕하세요?', '한글 문장 임베딩을 위한 BERT 모델입니다.']
embeddings = model.encode(sentences)
print(embeddings)

[[-0.37510458 -0.7733839   0.5927711  ...  0.57923526  0.3268346
  -0.65089655]
 [-0.03811135  0.20955126  0.02041095 ...  0.25595438  0.27676168
   0.06661468]]


In [4]:
df = pd.read_csv('data/부동산용어data.csv')
df.head(3)

,용어,챗봇
0,제1종 일반주거지역,"용도지역의 주거지역 중 일반주거지역의 하나로, 저층주택을 중심으로 편리한 주거환경을..."
1,자연녹지지역,국토의 계획 및 이용에 관한 법률에 의해 지정된 용도 중 녹지지역의 하나로 도시의 ...
2,제2종 일반주거지역,"용도지역의 주거지역 중 일반주거지역의 하나로, 중층주택을 중심으로 편리한 주거환경을..."


In [5]:
model.encode(df.용어[0])

array([-0.03030568, -0.43911374,  0.5236608 , -0.64376444,  0.5814941 ,
       -0.39388317,  0.15363362, -0.33766034,  0.40850464, -0.7201977 ,
        0.42764714, -0.15920812, -0.73644483, -0.06442916, -0.02017396,
        0.4882762 , -0.2067069 , -0.7067108 ,  0.4798753 , -0.51991445,
        0.4827748 , -0.20437962,  0.71420467, -0.22251585,  0.21981777,
        0.57903874, -0.6359728 ,  0.11202153,  0.75009036,  0.06690206,
        0.08908127,  0.21112823,  0.00339019, -1.0083327 ,  0.06868882,
        0.27920112, -0.49148422, -0.9997743 ,  0.22840479, -0.08925874,
        0.15952721, -0.47623083, -0.347791  ,  0.43106055,  0.22606367,
       -0.05086173, -0.40038592,  0.17954555,  0.28643024,  0.5950232 ,
        1.106266  , -0.86352855, -0.29279882,  0.09411384, -0.30828407,
       -0.27762768,  0.16063124, -0.21448314,  0.01563619, -0.6019861 ,
        0.3157862 ,  0.2260795 , -0.7054969 ,  0.3344907 , -0.3702947 ,
       -0.0996545 ,  0.24421705,  0.24373876, -0.5401106 , -0.13

In [6]:
df['embedding'] = pd.Series([[]] * len(df))     # dummy
df.shape

(1005, 3)

In [7]:
df.embedding = df.용어.map(lambda x: list(model.encode(x)))
df.head(3)

,용어,챗봇,embedding
0,제1종 일반주거지역,"용도지역의 주거지역 중 일반주거지역의 하나로, 저층주택을 중심으로 편리한 주거환경을...","[-0.030305676, -0.43911374, 0.5236608, -0.6437..."
1,자연녹지지역,국토의 계획 및 이용에 관한 법률에 의해 지정된 용도 중 녹지지역의 하나로 도시의 ...,"[0.10617465, -0.7045992, -0.11163301, 0.049178..."
2,제2종 일반주거지역,"용도지역의 주거지역 중 일반주거지역의 하나로, 중층주택을 중심으로 편리한 주거환경을...","[-0.081917636, -0.29992613, 0.53384227, -0.554..."


In [8]:
df.to_csv('data/부동용어_dataset.csv', index=False)

In [9]:
import json
wdf = pd.read_csv('data/부동용어_dataset.csv')
wdf.embedding = wdf.embedding.apply(json.loads)
wdf.head(3)

,용어,챗봇,embedding
0,제1종 일반주거지역,"용도지역의 주거지역 중 일반주거지역의 하나로, 저층주택을 중심으로 편리한 주거환경을...","[-0.030305676, -0.43911374, 0.5236608, -0.6437..."
1,자연녹지지역,국토의 계획 및 이용에 관한 법률에 의해 지정된 용도 중 녹지지역의 하나로 도시의 ...,"[0.10617465, -0.7045992, -0.11163301, 0.049178..."
2,제2종 일반주거지역,"용도지역의 주거지역 중 일반주거지역의 하나로, 중층주택을 중심으로 편리한 주거환경을...","[-0.081917636, -0.29992613, 0.53384227, -0.554..."


In [10]:
text = '전세'
embedding = model.encode(text)

In [11]:
wdf['유사도'] = wdf.embedding.map(lambda x: cosine_similarity([embedding],[x]).squeeze())

In [12]:
wdf.유사도.idxmax()

182

In [13]:
wdf.챗봇[wdf.유사도.idxmax()]

'전세란 집주인이 세 들어 사는 사람에게 집의 방 한 칸이나 집 전체를 빌려주고, 그 대가로 받는 돈이지만 세 사는 사람이 집을 나갈 때는 돌려주어야 하는 돈을 말한다. 사는 집에만 쓰는 말이며, 상가나 빌딩 등에는 쓰지 않는다. 전세는 주거관습으로 생겨 하나의 물권으로 발달된 것으로 한국에만 있는 특이한 주택 임대차 거래관행이다.이것은 집이 있는 사람과 집을 살 능력이 부족한 사람 간에 상호 이익이 부합하여 관행이 된 것이다. 특히 산업화사회에 접어들어 농촌인구의 급격한 도시진입은 주택 부족문제를 발생시키고 수급의 불균형을 초래하였으며, 도시에서 집 살 능력이 부족한 농촌 사람들은 셋집을 얻어야 했으며, 전세는 지속적으로 관행이 되었다.또 몫 돈의 전세금을 받는 사람은 금융권에서 대출받는 것이 어려웠으므로, 이것을 이용해 부족한 집 값을 채워 집을 사는데 활용하였다. 이처럼 이 거래관행은 세를 주는 자와 세를 얻는 자간에 자연스럽게 발전되었다. 이 전세관행은 집주인인 임대인과 세사는 임차인 모두에게 장점이 있다.임대인 입장에서 전세관행은 자금조달 수단으로 활용된다. 앞에 말한 것처럼 집살 때 부족한 자금을 이 전세자금으로 충당할 수 있고, 사업자금으로 활용하거나 금융기관에 맡겨 이자를 받을 수 있다. 임차인 입자에서는 강제 저축수단이 된다. 셋집을 얻을 때는 목돈이 들기 때문에 어떻게든 아끼고 절약하여 전셋돈을 마련한다.또 전세금으로 주고 나면 쓸 수도 없어서 돈이 낭비되지 않는다. 이처럼 강제 저축의 성격이 있는 이 자금은 나중에 내 집 마련자금으로 활용할 수 있다. 이러한 관행은 외환위기 이후 경제가 침체되면서 이자율이 낮아지자 셋 집주인들은 셋 돈을 활용하기 어려워 전세금을 보증금 일부, 월세 일부로 환산하여 받기 시작하였으며, 또 세입자도 목돈을 적게 들이는 이 방식을 받아들이게 되었다.그래서 현재는 전세관행이 보증금부 월세 관행으로 전이되는 과정에 있다. 또 서구나 일본처럼 보증금 없이 월세만 내고 임대차하는 관행도 나타나 이용되고 있다. 비제도권 

In [24]:
text = '매매'
embedding = model.encode(text)
wdf['유사도'] = wdf.embedding.map(lambda x: cosine_similarity([embedding], [x]).squeeze())
answer = wdf.챗봇[wdf.유사도.idxmax()]
answer

'타인으로부터 무상으로 재산을 취득한 자가 납부하는 국세로서 직접세이며 재산세로 분류되는 조세를 말한다. 증여가 증여자의 생전에 이루어진다는 점에서 상속세와 다르다.① 증여세의 납세의무자는 개인과 법인 및 연체납세의무자로 구분한다.② 증여세의 납세지는 수증자의 주소지 또는 거주지를 관할하는 세무서로 한다.③ 증여재산의 범위는 수증자가 수증 당시 국내에 주소를 둔 경우에는 증여받은 재산 전부에 증여세를 부과하나 수증자가 수증 당시 국내에 주소를 두지 아니한 경우에는 증여받은 재산 중 국내에 있는 재산에 대하여 부과한다.④ 증여세의 과세표준은 증여재산의 과세가액에서 증여재산공제를 차감하여 계산한다.증여세를 자진 신고 후 자진 납부하는 경우에는 신고 세액공제를 받을 수 있으며, 증여지를 직접 경작하는 경우에도 감면을 받을 수 있다. 반면에 부당한 방법으로 신고를 하지 않거나 납세를 하지 않은 경우에는 신고불성실가산세액과 납부불성실가산세액을 부과 받을 수 있다.'